In [34]:
import json
import pprint

In [35]:
!if [ ! -f HumanEval.jsonl.gz ]; then wget https://github.com/openai/human-eval/raw/master/data/HumanEval.jsonl.gz; fi
!if [ ! -f HumanEval.jsonl ]; then gunzip HumanEval.jsonl.gz;fi

--2023-11-29 17:16:51--  https://github.com/openai/human-eval/raw/master/data/HumanEval.jsonl.gz
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/openai/human-eval/master/data/HumanEval.jsonl.gz [following]
--2023-11-29 17:16:51--  https://raw.githubusercontent.com/openai/human-eval/master/data/HumanEval.jsonl.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44877 (44K) [application/octet-stream]
Saving to: ‘HumanEval.jsonl.gz’

HumanEval.jsonl.gz  100%[===================>]  43.83K  --.-KB/s    in 0.01s   

2023-11-29 17:16:51 (3.68 MB/s) - ‘HumanEval.jsonl.gz’ saved [44877/44877]



In [36]:
with open("HumanEval.jsonl", "r") as f:
  human_eval = [json.loads(line) for line in f]

print(human_eval[0])
print("Size: ", len(human_eval))


{'task_id': 'HumanEval/0', 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n', 'entry_point': 'has_close_elements', 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n', 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0

In [37]:
human_eval[0].keys()


dict_keys(['task_id', 'prompt', 'entry_point', 'canonical_solution', 'test'])

In [38]:
import random

N_CONVERSASTIONS = 40
MIN_CONVERSATION_LENGTH = 3
TOTAL_WORD_LENGTH_LIMIT = 2048 * 0.75
SEED = 123

random.seed(SEED)
code_conversations = []

for i in range(N_CONVERSASTIONS):
  convo = random.sample(human_eval, MIN_CONVERSATION_LENGTH)
  convo = [interaction['prompt'] for interaction in convo]
  total_length = sum([len(prompt) for prompt in convo])
  if total_length < TOTAL_WORD_LENGTH_LIMIT:
    code_conversations.append(convo)


In [39]:
len(code_conversations)

28

In [40]:
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(code_conversations[0][0])


('from typing import List, Tuple\n'
 '\n'
 '\n'
 'def rolling_max(numbers: List[int]) -> List[int]:\n'
 '    """ From a given list of integers, generate a list of rolling maximum '
 'element found until given moment\n'
 '    in the sequence.\n'
 '    >>> rolling_max([1, 2, 3, 2, 3, 4, 2])\n'
 '    [1, 2, 3, 3, 3, 4, 4]\n'
 '    """\n')


In [41]:
with open("code_conversations.json", "w") as f:
  json.dump(code_conversations, f, indent=4)
